In [1]:
import requests
import pandas as pd
import re

def extract_album_id(album_link):
    match = re.search(r'/album/.*?/([\w-]+)\.html', album_link or "")
    return match.group(1) if match else "N/A"

def determine_album_type(track_count):
    if track_count == 1:
        return "Single"
    elif 2 <= track_count <= 6:
        return "EP"
    else:
        return "Regular"

def fetch_artist_songs(artist_name):
    url = f"http://localhost:5000/api/artistsongs?name={artist_name}"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        songs = data.get("songs", [])
        
        records = []
        album_tracks = {}
        
        for song in songs:
            album_id = extract_album_id(song.get("albumLink", "") or "")
            album_name = song.get("album", song.get("title", "N/A"))
            tracklist = song.get("tracklist", [])
            
            if not tracklist:
                tracklist = [{"title": song.get("title", "N/A"), "link": song.get("link", "N/A")}]  # Nếu không có tracklist, tạo dữ liệu từ bài hát chính
            
            if album_id not in album_tracks:
                album_tracks[album_id] = len(tracklist)
            
            for track in tracklist:
                records.append({
                    "album_id": album_id,
                    "album_name": album_name,
                    "tracklist": track.get("title", "N/A"),
                    "release_date": song.get("releaseDate", "Unknown"),
                    "provided_by": song.get("providedBy", "Unknown"),
                    "featured_artists": song.get("featuredArtists", "Unknown"),
                    "album_artist": song.get("albumOwner", "Unknown"),
                    "ZingMP3": track.get("link", "N/A")
                })
        
        df = pd.DataFrame(records)
        df = df.drop_duplicates(subset=["album_id", "album_name", "tracklist"])
        df = df.sort_values(by=["album_name", "tracklist"], ascending=[True, True])
        
        # Thêm cột phân loại album
        df["album_type"] = df["album_id"].map(lambda x: determine_album_type(album_tracks.get(x, 1)))
        
        return df
    else:
        print("Error fetching data:", response.status_code)
        return None

artist_name = 'Rapper-Ngan'
df = fetch_artist_songs(artist_name)



In [2]:
df.to_excel(f'{artist_name}_songZingMP3.xlsx')
df.head()

,album_id,album_name,tracklist,release_date,provided_by,featured_artists,album_artist,ZingMP3,album_type
32,60BODZB8,9X (Single),9X,04/05/2020,MIXUS,"DLblack, Ngắn, Mêk Team","DLblack, Ngắn, Mêk Team",https://zingmp3.vn/bai-hat/9X-DLblack-Ngan-Mek...,Single
20,Z6DU0B8O,9X (Single),Trở Về,2019,MIXUS,"Ngắn, Hải Phòng Sound","Ngắn, Hải Phòng Sound",https://zingmp3.vn/bai-hat/Tro-Ve-Ngan-Hai-Pho...,Single
53,6909U6OC,Bình Dân (Remix Version) (Single),Bình Dân (Remix Version),10/09/2021,MIXUS,"Ngắn, Monkieq","Ngắn, Monkieq",https://zingmp3.vn/bai-hat/Binh-Dan-Remix-Vers...,Single
62,6BUBDBAI,Bụi Thiên Thần (EP),Cuộc Đời Này Ngắn,22/07/2022,LOOPS Music,Ngắn,Ngắn,https://zingmp3.vn/bai-hat/Cuoc-Doi-Nay-Ngan-N...,EP
66,6BUBDBAI,Bụi Thiên Thần (EP),Người Nghệ Sĩ Cô Đơn,22/07/2022,LOOPS Music,Ngắn,Ngắn,https://zingmp3.vn/bai-hat/Nguoi-Nghe-Si-Co-Do...,EP


In [3]:
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials

# Hàm xác định loại album
def determine_album_type(track_count):
    if track_count <= 3:
        return "Single"
    elif 4 <= track_count <= 6:
        return "EP"
    else:
        return "Regular"

# Hàm lấy tất cả bài hát của nghệ sĩ
def get_artist_tracks_all(artist_name):
    client_id = "c9ef6e4338814489867afc9956093213"
    client_secret = "a449438b21d44525a68a575cf67c8cfd"
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

    # 🔎 Tìm nghệ sĩ theo tên
    results = sp.search(q=artist_name, type='artist', limit=1)
    if not results['artists']['items']:
        print("Không tìm thấy nghệ sĩ.")
        return pd.DataFrame()

    artist_id = results['artists']['items'][0]['id']

    track_data = []
    seen_tracks = set()
    album_tracks_count = {}


    offset = 0
    while True:
        albums = sp.artist_albums(artist_id, album_type='album,single', limit=50, offset=offset)
        if not albums['items']:
            break
        for album in albums['items']:
            album_id = album['id']
            album_name = album['name']
            album_release_date = album.get('release_date', 'Unknown')
            album_owner = album['artists'][0]['name']

            album_info = sp.album(album_id)
            label = album_info.get('label', 'Unknown')

           
            tracks = sp.album_tracks(album_id)['items']
            album_tracks_count[album_name] = len(tracks)  # Ghi lại số lượng bài hát

            for track in tracks:
                track_id = track['id']
                if track_id in seen_tracks:
                    continue
                seen_tracks.add(track_id)

                track_title = track['name']
                link_spotify = track['external_urls']['spotify']
                featured_artists = [artist['name'] for artist in track['artists'] if artist['id'] != artist_id]
                featured_artists = ", ".join(featured_artists) if featured_artists else "None"

                # Xử lý ngày phát hành
                try:
                    album_release_date = pd.to_datetime(album_release_date, errors='coerce').strftime("%d/%m/%Y")
                except:
                    album_release_date = "Unknown"

                track_data.append([album_name, track_title, album_release_date, featured_artists, album_owner, label, link_spotify])
        offset += 50  

   
    top_tracks = sp.artist_top_tracks(artist_id, country="US")['tracks']
    for track in top_tracks:
        track_id = track['id']
        if track_id in seen_tracks:
            continue
        seen_tracks.add(track_id)

        track_title = track['name']
        link_spotify = track['external_urls']['spotify']
        album = track['album']
        album_name = album['name']
        album_release_date = album.get('release_date', 'Unknown')
        album_owner = album['artists'][0]['name']
        featured_artists = [artist['name'] for artist in track['artists'] if artist['id'] != artist_id]
        featured_artists = ", ".join(featured_artists) if featured_artists else "None"

        # 🔹 Lấy thông tin label của album
        album_info = sp.album(album['id'])
        label = album_info.get('label', 'Unknown')

        try:
            album_release_date = pd.to_datetime(album_release_date, errors='coerce').strftime("%d/%m/%Y")
        except:
            album_release_date = "Unknown"

        track_data.append([album_name, track_title, album_release_date, featured_artists, album_owner, label, link_spotify])
        album_tracks_count[album_name] = album_tracks_count.get(album_name, 0) + 1  # Cập nhật số bài hát


    columns = ["album_name", "tracklist", "release_date", "featured_artists", "album_owner_", "provided_by", "Link_Spotify"]
    df = pd.DataFrame(track_data, columns=columns)

    df['album_type'] = df['album_name'].map(lambda x: determine_album_type(album_tracks_count.get(x, 0)))
    df = df.sort_values(by=["album_name", "tracklist"], ascending=[True, True])
 

    return df

# Chạy chương trình
if __name__ == "__main__":
    artist_name_Spotify = 'Ngắn'
    df_tracks = get_artist_tracks_all(artist_name_Spotify)



    

C:\Users\mdtruong\AppData\Local\Temp\ipykernel_10504\2561442202.py:64: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  album_release_date = pd.to_datetime(album_release_date, errors='coerce').strftime("%d/%m/%Y")


In [4]:

df_tracks.to_excel(f"{artist_name_Spotify}_songSpotify.xlsx", index=False)
df_tracks.head()

,album_name,tracklist,release_date,featured_artists,album_owner_,provided_by,Link_Spotify,album_type
24,120Bpm (ft. Long Mộng Gà),120Bpm (ft. Long Mộng Gà),10/03/2023,Antoneus Maximus,Ngắn,MaiDao Music,https://open.spotify.com/track/3QSw9fOs1Qe14tg...,Single
62,9X,9X,04/05/2020,"DLBlack, Mêk Team",DLBlack,Zing MP3,https://open.spotify.com/track/2Gj8BVWfiqHTykY...,Single
50,Bình Dân,Bình Dân,03/09/2021,None,Ngắn,Yin Yang Media,https://open.spotify.com/track/3dhDxLqZScUdYDx...,Single
51,Bình Dân,Bình Dân - Beat,09/03/2021,None,Ngắn,Yin Yang Media,https://open.spotify.com/track/0eof9V6sNRlPtiQ...,Single
30,Bụi Thiên Thần,Cuộc Đời Này Ngắn,27/07/2022,None,Ngắn,LOOPS Music,https://open.spotify.com/track/3UmZvOmW9QM7W1F...,EP


In [5]:
import pandas as pd
import re
from fuzzywuzzy import process

def is_remix(title):
    """Kiểm tra xem tiêu đề có phải bản remix hay không"""
    return bool(re.search(r'\b(remix|version|edit)\b', str(title), re.IGNORECASE))

def find_best_match(title, choices, threshold=90):
    """Tìm album phù hợp, ưu tiên đúng loại Remix/Gốc"""
    is_title_remix = is_remix(title)
    
    # Lọc danh sách theo bản remix/gốc tương ứng
    filtered_choices = [c for c in choices if is_remix(c) == is_title_remix]
    match_list = filtered_choices if filtered_choices else choices  # Nếu không có, thử toàn bộ
    
    match, score = process.extractOne(title, match_list)
    return match if score >= threshold else None

# Đọc dữ liệu từ file (giả sử df_tracks là danh sách bài hát Spotify, df là dữ liệu từ ZingMP3)


# Tạo từ điển album từ ZingMP3
album_titles_dict = {title: title for title in df["album_name"]}

# Tìm album phù hợp nhất giữa Spotify và ZingMP3
df_tracks["best_match_album"] = df_tracks["album_name"].apply(lambda x: find_best_match(x, album_titles_dict.keys()))

# Gộp dữ liệu dựa trên album_name khớp được
df_merged = pd.merge(df_tracks, df, left_on="best_match_album", right_on="album_name", how="outer", suffixes=("_Spotify", "_ZingMP3"))

# Gán giá trị từ hai nguồn dữ liệu
df_merged = df_merged.assign(
    album_name=df_merged["album_name_Spotify"].combine_first(df_merged["album_name_ZingMP3"]),
    track_title=df_merged["tracklist_Spotify"].combine_first(df_merged["tracklist_ZingMP3"]),
    album_type=df_merged["album_type_Spotify"].fillna(df_merged["album_type_ZingMP3"]),
    album_owner_spotify=df_merged["album_owner_Spotify"].combine_first(df_merged["album_artist"]),
    release_date_spotify=df_merged["release_date_Spotify"],
    release_date_zingmp3=df_merged["release_date_ZingMP3"],
    featured_artists_spotify=df_merged["featured_artists_Spotify"],
    featured_artists_zingmp3=df_merged["featured_artists_ZingMP3"],
    label_spotify=df_merged["provided_by_Spotify"],
    label_zingmp3=df_merged["provided_by_ZingMP3"],
    album_id_zingmp3=df_merged["album_id"],
    link_spotify=df_merged["Spotify"],
    link_zingmp3=df_merged["ZingMP3"]
)

# Xóa cột không cần thiết
df_final = df_merged.drop(columns=["best_match_album", "album_name_Spotify", "album_name_ZingMP3"])

# Đổi tên cột
df_final = df_final.rename(columns={
    "album_name": "Tên album",
    "album_owner_spotify": "Nghệ sĩ sở hữu album (Spotify)",
    "album_owner_zingmp3": "Nghệ sĩ sở hữu album (ZingMP3)",
    "album_type": "Loại album",
    "track_title": "Danh sách bài hát",
    "release_date_spotify": "Ngày phát hành trên Spotify",
    "release_date_zingmp3": "Ngày phát hành trên ZingMP3",
    "featured_artists_spotify": "Nghệ sĩ tham gia (Spotify)",
    "featured_artists_zingmp3": "Nghệ sĩ tham gia (ZingMP3)",
    "label_spotify": "Cung cấp bởi (Spotify)",
    "label_zingmp3": "Cung cấp bởi (ZingMP3)",
    "album_id_zingmp3": "Mã album ZingMP3",
    "link_spotify": "Link Spotify",
    "link_zingmp3": "Link ZingMP3"
})

# Sắp xếp dữ liệu
df_final = df_final.sort_values(by=["Tên album", "Mã album ZingMP3"], ascending=[True, True])

# Loại bỏ trùng lặp dựa trên album_name và tracklist
df_final = df_final.drop_duplicates(subset=["Tên album", "Danh sách bài hát"], keep="first")

# Xuất file Excel sau khi lọc trùng
df_final.to_excel("Merged_AlbumsZingMp3_Spot.xlsx", index=False)

print("Xuất file hoàn tất!")
df_final.head()


c:\Users\mdtruong\AppData\Local\Programs\Python\Python311\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


NameError: name 'df_tracks' is not defined

In [110]:
print(df_tracks.columns)

RangeIndex(start=0, stop=0, step=1)


In [5]:
import pandas as pd
import re

# Hàm chuẩn hóa tên album
def normalize_album_name(album_name):
    return re.sub(r"\s*\(.*?\)", "", album_name).strip()

# Chọn các cột cần thiết từ cả hai nguồn
df_tracks = df_tracks[
    [
        'album_name', 'tracklist', 'release_date', 'featured_artists',
       'album_owner_', 'provided_by', 'Link_Spotify', 'album_type',
      
    ]
]

df = df[
    [
      'album_id', 'album_name', 'tracklist', 'release_date', 'provided_by',
       'featured_artists', 'album_artist', 'ZingMP3', 'album_type'
    ]
]

# Chuẩn hóa tên album để đồng bộ dữ liệu
df_tracks["album_name"] = df_tracks["album_name"].apply(normalize_album_name)
df["album_name"] = df["album_name"].apply(normalize_album_name)

# Gộp dữ liệu dựa trên album_name, tracklist, album_type
df_merged = pd.merge(df_tracks, df, on=["album_name", "tracklist", "album_type"], how="outer", suffixes=("_Spotify", "_ZingMP3"))


# Đổi tên cột
df_ = df_merged.rename(columns={
    "album_name_Spotify": "album_name",
    "tracklist": "tracklist(danh sách bài hát)",
    "featured_artists_Spotify": "Song artist(nghệ sĩ tham gia bài hát)(Spotify)",
    "album_owner_Spotify": "Album artist (nghệ sĩ sở hữu album)*(Spotify)",
    "release_date_Spotify": "Ngày phát hành trên Spotify",
    "provided_by": "Cung cấp bởi(ZingMP3)",
    "label": "Cung cấp bởi(Spotify)",
    "featured_artists_ZingMP3": "Song artist(nghệ sĩ tham gia bài hát)(ZingMP3)",
    "album_owner_ZingMP3": "Album artist (nghệ sĩ sở hữu album)*(ZingMP3)",
    "release_date_ZingMP3": "Ngày phát hành trên ZingMP3",
    "linkzingmp3": "ZingMP3",
    "Link_Spotify": "Spotify",
    "album_id": "Mã định danh album zingmp3"
})


desired_columns = [
    "album_name",
    "album_owner_",  # Xác nhận lại nếu đây là 'Album artist'
   
    "album_type",
    "tracklist(danh sách bài hát)",
    "Ngày phát hành trên Spotify",
    "Ngày phát hành trên ZingMP3",
    "Song artist(nghệ sĩ tham gia bài hát)(Spotify)",
    "Song artist(nghệ sĩ tham gia bài hát)(ZingMP3)",
    "provided_by_ZingMP3", 
    "provided_by_Spotify", 
    "Mã định danh album zingmp3",
    "ZingMP3",
    "Spotify"
]

df_ = df_[desired_columns]




df_.to_excel("song_ZingMP3+Spotify.xlsx", index=False)

# Hiển thị kết quả

In [6]:
df_

,album_name,album_owner_,album_type,tracklist(danh sách bài hát),Ngày phát hành trên Spotify,Ngày phát hành trên ZingMP3,Song artist(nghệ sĩ tham gia bài hát)(Spotify),Song artist(nghệ sĩ tham gia bài hát)(ZingMP3),provided_by_ZingMP3,provided_by_Spotify,Mã định danh album zingmp3,ZingMP3,Spotify
0,120Bpm,Ngắn,Single,120Bpm (ft. Long Mộng Gà),10/03/2023,NaN,Antoneus Maximus,NaN,NaN,MaiDao Music,NaN,NaN,https://open.spotify.com/track/3QSw9fOs1Qe14tg...
1,9X,DLBlack,Single,9X,04/05/2020,04/05/2020,"DLBlack, Mêk Team","DLblack, Ngắn, Mêk Team",MIXUS,Zing MP3,60BODZB8,https://zingmp3.vn/bai-hat/9X-DLblack-Ngan-Mek...,https://open.spotify.com/track/2Gj8BVWfiqHTykY...
2,9X,NaN,Single,Trở Về,NaN,2019,NaN,"Ngắn, Hải Phòng Sound",MIXUS,NaN,Z6DU0B8O,https://zingmp3.vn/bai-hat/Tro-Ve-Ngan-Hai-Pho...,NaN
3,Bình Dân,Ngắn,Single,Bình Dân,03/09/2021,NaN,None,NaN,NaN,Yin Yang Media,NaN,NaN,https://open.spotify.com/track/3dhDxLqZScUdYDx...
4,Bình Dân,NaN,Single,Bình Dân (Remix Version),NaN,10/09/2021,NaN,"Ngắn, Monkieq",MIXUS,NaN,6909U6OC,https://zingmp3.vn/bai-hat/Binh-Dan-Remix-Vers...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,Đơn Phương Yêu Một Người,NaN,Single,Đơn Phương Yêu Một Người (Acoustic),NaN,06/12/2023,NaN,"Longg, Ngắn",LOOPS Music,NaN,6BEIUO69,https://zingmp3.vn/bai-hat/Don-Phuong-Yeu-Mot-...,NaN
101,Đơn Phương Yêu Một Người,Longg,Single,Đơn Phương Yêu Một Người - Acoustic,06/12/2023,NaN,Longg,NaN,NaN,LOOPS Music,NaN,NaN,https://open.spotify.com/track/06qRaDxPsCUnZzM...
102,Đường Về,DLBlack,Single,Đường Về,01/01/2020,2019,"DLBlack, JoyBlue","DLblack, Ngắn, JoyBlue",MIXUS,Zing MP3,ZUBZ707F,https://zingmp3.vn/bai-hat/Duong-Ve-DLblack-Ng...,https://open.spotify.com/track/3Ua7AqRWM54CTyq...
103,ĐẬM SÂU,Lor,Single,ĐẬM SÂU,07/10/2022,07/10/2022,Lor,"Lor, Ngắn",Ingrooves Music Group,M Music Records,6B6OECUF,https://zingmp3.vn/bai-hat/DAM-SAU-Lor-Ngan/Z6...,https://open.spotify.com/track/0frxC5D0uIjToqh...


In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd

def search_artist_nhaccuatui(driver, artist_name):
    """Tìm nghệ sĩ trên NhacCuaTui và lấy link trang nghệ sĩ."""
    driver.get("https://www.nhaccuatui.com/")
    time.sleep(3)

    try:
        search_box = driver.find_element(By.CSS_SELECTOR, "input#txtSearch")
        search_box.click()
        time.sleep(1)

        search_box.send_keys(artist_name)
        time.sleep(3)

        suggestions = driver.find_elements(By.CSS_SELECTOR, ".info-search .qsItem a")
        for suggestion in suggestions:
            if "nghe-si" in suggestion.get_attribute("href"):
                return suggestion.get_attribute("href")
    except Exception as e:
        print(f"Lỗi khi tìm nghệ sĩ {artist_name}: {e}")
    return None

def get_artist_songs(driver, artist_song_url):
    """Lấy danh sách bài hát của nghệ sĩ từ tất cả các trang."""
    driver.get(artist_song_url)
    time.sleep(3)
    song_list = []
    while True:
        song_blocks = driver.find_elements(By.CSS_SELECTOR, ".box-content-music-list .info_song")
        for song_block in song_blocks:
            try:
                song_name = song_block.find_element(By.CSS_SELECTOR, "h3 a").text
                song_link = song_block.find_element(By.CSS_SELECTOR, "h3 a").get_attribute("href")
                song_list.append(("N/A", song_name, song_link))
            except Exception as e:
                print(f"Lỗi khi lấy bài hát: {e}")
        try:
            next_button = driver.find_element(By.CSS_SELECTOR, "a.number[rel='next']")
            next_link = next_button.get_attribute("href")
            if next_link:
                driver.get(next_link)
                time.sleep(3)
            else:
                break
        except:
            break
    return song_list

def get_artist_albums(driver, artist_album_url):
    """Lấy danh sách Album của nghệ sĩ."""
    driver.get(artist_album_url)
    time.sleep(3)

    album_list = []
    while True:
        album_blocks = driver.find_elements(By.CSS_SELECTOR, ".box-left-album a.box_absolute")
        for album in album_blocks:
            try:
                album_link = album.get_attribute("href")
                album_list.append(album_link)
            except Exception as e:
                print(f"Lỗi khi lấy album: {e}")
        try:
            next_button = driver.find_element(By.CSS_SELECTOR, "a.number[rel='next']")
            next_link = next_button.get_attribute("href")
            if next_link:
                driver.get(next_link)
                time.sleep(3)
            else:
                break
        except:
            break
    return album_list

def get_album_songs(driver, album_url):
    """Lấy danh sách bài hát trong Album."""
    driver.get(album_url)
    time.sleep(3)
    try:
        album_name = driver.find_element(By.CSS_SELECTOR, ".name_title").text
        song_elements = driver.find_elements(By.CSS_SELECTOR, "li[id^='itemSong_']")
        album_songs = [(album_name,
                        s.find_element(By.CSS_SELECTOR, "meta[itemprop='name']").get_attribute("content"),
                        s.find_element(By.CSS_SELECTOR, "meta[itemprop='url']").get_attribute("content"))
                        for s in song_elements]
        return album_songs
    except Exception as e:
        print(f"Lỗi khi lấy bài hát từ album {album_url}: {e}")
    return []

driver = webdriver.Chrome()
artist = "Ngắn"

artist_page = search_artist_nhaccuatui(driver, artist)
if artist_page:
    print(f"Trang nghệ sĩ: {artist_page}")

    artist_song_page = artist_page.replace(".html", ".bai-hat.html")
    artist_album_page = artist_page.replace(".html", ".playlist.html")

    artist_songs = get_artist_songs(driver, artist_song_page)
    print(pd.DataFrame(artist_songs, columns=["Album", "Tên bài hát", "Link bài hát"]))
    
    albums = get_artist_albums(driver, artist_album_page)
    print(f"Tìm thấy {len(albums)} album.")
    artist_songs = [(song_name, song_name, song_link) for _, song_name, song_link in artist_songs]
    all_songs = artist_songs[:]
    for album_link in albums:
        album_songs = get_album_songs(driver, album_link)
        all_songs.extend(album_songs)

    df_nct = pd.DataFrame(all_songs, columns=["album_name", "tracklist(danh sách bài hát)", "Link bài hát"])
    df_nct.to_excel(f"{artist}_NhacCuaTui_Songs.xlsx", index=False)
    print(f"Lấy được {len(all_songs)} bài hát của {artist}, lưu vào file Excel!")
else:
    print("Không tìm thấy nghệ sĩ.")
driver.quit()


ReadTimeoutError: HTTPConnectionPool(host='localhost', port=50730): Read timed out. (read timeout=120)

In [26]:
df_nct

,album_name,tracklist(danh sách bài hát),Link bài hát
0,NGẮN | Trăng Tròn - phiên bản quẩy tung nóc đê...,NGẮN | Trăng Tròn - phiên bản quẩy tung nóc đê...,https://www.nhaccuatui.com/bai-hat/ngan-trang-...
1,TÌM HÀNH TINH KHÁC | VŨ CÁT TƯỜNG x NGẮN,TÌM HÀNH TINH KHÁC | VŨ CÁT TƯỜNG x NGẮN,https://www.nhaccuatui.com/bai-hat/tim-hanh-ti...
2,NGẮN | BÌNH DÂN | MV Official,NGẮN | BÌNH DÂN | MV Official,https://www.nhaccuatui.com/bai-hat/ngan-binh-d...
3,Em Ở Nơi Đâu - Ngắn aka Lil Shadow Hải Phòng S...,Em Ở Nơi Đâu - Ngắn aka Lil Shadow Hải Phòng S...,https://www.nhaccuatui.com/bai-hat/em-o-noi-da...
4,NGẮN | NƠI TAO SINH RA | MV OFFICIAL | Prod - ...,NGẮN | NƠI TAO SINH RA | MV OFFICIAL | Prod - ...,https://www.nhaccuatui.com/bai-hat/ngan-noi-ta...
...,...,...,...
89,"em là em bé iu (SS x DJ AM Remix) - Olew, Ngắn",em là em bé iu (SS x DJ AM Remix),https://www.nhaccuatui.com/bai-hat/em-la-em-be...
90,"em là em bé iu (Single) - Olew, Ngắn",em là em bé iu,https://www.nhaccuatui.com/bai-hat/em-la-em-be...
91,"Vườn Hoa Con Cá (Lofi) - Olew, Ngắn, meChill",Vườn Hoa Con Cá (Lofi),https://www.nhaccuatui.com/bai-hat/vuon-hoa-co...
92,"Cạm Bẫy (Single) - Dick, Ngắn, Ziog",Cạm Bẫy,https://www.nhaccuatui.com/bai-hat/cam-bay-dic...


In [7]:
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# import time
# import pandas as pd

# def search_nhaccuatui(artist_name):
#     driver = webdriver.Chrome()
    
#     search_url_songs = f"https://www.nhaccuatui.com/tim-kiem/bai-hat?q={artist_name}&b=keyword&l=tat-ca&s=default"
#     search_url_playlists = f"https://www.nhaccuatui.com/tim-kiem/playlist?q={artist_name}&b=keyword&l=tat-ca&s=default"
    
#     driver.get(search_url_songs)
#     time.sleep(3)
    
#     song_list = []
    
#     while True:
#         songs = driver.find_elements(By.CSS_SELECTOR, ".box_info h3.title_song a")
#         artists = driver.find_elements(By.CSS_SELECTOR, ".box_info h4.singer_song")
        
#         for song, artist in zip(songs, artists):
#             artist_names = ", ".join([a.text for a in artist.find_elements(By.TAG_NAME, "a")])
#             song_list.append((song.text, song.get_attribute("href"), artist_names))
        
#         try:
#             next_button = driver.find_element(By.CSS_SELECTOR, "a.number[rel='next']")
#             driver.execute_script("arguments[0].click();", next_button)
#             time.sleep(3)
#         except:
#             break
    
#     driver.get(search_url_playlists)
#     time.sleep(3)
    
#     checked_playlists = set()
#     playlist_links = []
    
#     while True:
#         try:
#             playlists = driver.find_elements(By.CSS_SELECTOR, ".box_info h3.title_song a")
#             for p in playlists:
#                 link = p.get_attribute("href")
#                 if link and link not in checked_playlists:
#                     playlist_links.append(link)
#                     checked_playlists.add(link)
#         except Exception as e:
#             print(f"Lỗi khi lấy danh sách playlist: {e}")
        
#         try:
#             next_button = driver.find_element(By.CSS_SELECTOR, "a.number[rel='next']")
#             driver.execute_script("arguments[0].click();", next_button)
#             time.sleep(3)
#         except:
#             break
    
#     playlist_list = []
    
#     for link in playlist_links:
#         retry = 3
#         while retry > 0:
#             try:
#                 driver.get(link)
#                 time.sleep(3)
                
#                 album_name = driver.find_element(By.CSS_SELECTOR, ".name_title").text
                
#                 song_elements = driver.find_elements(By.CSS_SELECTOR, "li[id^='itemSong_']")
#                 playlist_songs = [(s.find_element(By.CSS_SELECTOR, "meta[itemprop='name']").get_attribute("content"),
#                                    s.find_element(By.CSS_SELECTOR, "meta[itemprop='url']").get_attribute("content"))
#                                    for s in song_elements]
                
#                 playlist_list.append((album_name, playlist_songs, link))
#                 break
#             except Exception as e:
#                 print(f"Lỗi khi lấy dữ liệu playlist {link}: {e}")
#                 driver.refresh()
#                 time.sleep(3)
#                 retry -= 1
    
#     driver.quit()
#     return song_list, playlist_list

# if __name__ == "__main__":
#     artist = "MCK"
#     songs, playlists = search_nhaccuatui(artist)
    
#     data = []
    
#     for song_title, song_link, _ in songs:
#         data.append([song_title, song_title, song_link])
    
#     for album_name, playlist_songs, link in playlists:
#         for title, song_link in playlist_songs:
#             data.append([album_name, title, song_link])
    
#     df_nct = pd.DataFrame(data, columns=["album_name", "tracklist(danh sách bài hát)", "Nhaccuatui"])
    
#     # Lưu vào file CSV
#     df_nct.to_excel(f"{artist}_NhacCuaTui.xlsx", index=False)
    
                


In [ ]:
import pandas as pd
from fuzzywuzzy import process

def normalize_text(text):
    """Chuẩn hóa văn bản: loại bỏ khoảng trắng thừa, chữ thường, bỏ ký tự đặc biệt."""
    return text.strip().lower().title() if isinstance(text, str) else text  

# 🗂 Chuẩn hóa dữ liệu đầu vào
df_final['norm_album_name'] = df_final['album_name'].apply(normalize_text)
df_nct['norm_album_name'] = df_nct['album_name'].apply(normalize_text)
df_final['norm_tracklist'] = df_final['tracklist(danh sách bài hát)'].apply(normalize_text)
df_nct['norm_tracklist'] = df_nct['tracklist(danh sách bài hát)'].apply(normalize_text)

def find_best_match(value, choices, threshold=85):  
    """Khớp dữ liệu mờ để tìm tên giống nhất"""
    if pd.isna(value) or not value:  
        return None, 0  
    match, score = process.extractOne(value, choices) if choices else (None, 0)
    return (match if score >= threshold else None), score  

# 🛠 Tạo danh sách để fuzzy match
album_choices = df_nct['norm_album_name'].dropna().unique().tolist()
tracklist_choices = df_nct['norm_tracklist'].dropna().unique().tolist()

# 🔍 Áp dụng fuzzy matching
df_final[['matched_album', 'album_score']] = df_final['norm_album_name'].apply(
    lambda x: pd.Series(find_best_match(x, album_choices))
)
df_final[['matched_tracklist', 'tracklist_score']] = df_final['norm_tracklist'].apply(
    lambda x: pd.Series(find_best_match(x, tracklist_choices))
)

# 🔗 Gộp dữ liệu dựa trên kết quả matching
df_merged = df_final.merge(
    df_nct, left_on='matched_album', right_on='norm_album_name', how='left', suffixes=('_final', '_nct')
)

# 🛠 Đổi tên cột hợp lý
df_merged.rename(columns={
    'album_name_final': 'album_name',
    'tracklist(danh sách bài hát)_final': 'tracklist(danh sách bài hát)',
    'Link bài hát': 'Nhaccuatui'
}, inplace=True)

# 📌 Chọn và sắp xếp lại thứ tự cột
column_order = [
    "album_name", "tracklist(danh sách bài hát)",
    "Album artist (nghệ sĩ sở hữu album)*(Spotify)", "Album artist (nghệ sĩ sở hữu album)*(ZingMP3)",
    "album_type", "Ngày phát hành trên Spotify", "Ngày phát hành trên ZingMP3",
    "Song artist(nghệ sĩ tham gia bài hát)(Spotify)", "Song artist(nghệ sĩ tham gia bài hát)(ZingMP3)",
    "Cung cấp bởi(ZingMP3)", "Cung cấp bởi(Spotify)",
    "Mã định danh album zingmp3", "ZingMP3", "Spotify", "Nhaccuatui"
]

existing_columns = [col for col in column_order if col in df_merged.columns]
df_merged = df_merged[existing_columns]
df_merged.drop_duplicates(subset=["album_name", "tracklist(danh sách bài hát)"], inplace=True)

df_merged.head()




,album_name,tracklist(danh sách bài hát),Album artist (nghệ sĩ sở hữu album)*(Spotify),Album artist (nghệ sĩ sở hữu album)*(ZingMP3),album_type,Ngày phát hành trên Spotify,Ngày phát hành trên ZingMP3,Song artist(nghệ sĩ tham gia bài hát)(Spotify),Song artist(nghệ sĩ tham gia bài hát)(ZingMP3),Cung cấp bởi(ZingMP3),Cung cấp bởi(Spotify),Mã định danh album zingmp3,ZingMP3,Spotify,Nhaccuatui
0,120bpm (ft. long mộng gà),120Bpm (ft. Long Mộng Gà),Ngắn,NaN,Single,10/03/2023,NaN,Antoneus Maximus,NaN,NaN,MaiDao Music,NaN,NaN,https://open.spotify.com/track/3QSw9fOs1Qe14tg...,https://www.nhaccuatui.com/bai-hat/120bpm-ngan...
1,9x,9X,DLBlack,"DLblack, Ngắn, Mêk Team",Single,04/05/2020,04/05/2020,"DLBlack, Mêk Team","DLblack, Ngắn, Mêk Team",MIXUS,Zing MP3,60BODZB8,https://zingmp3.vn/bai-hat/9X-DLblack-Ngan-Mek...,https://open.spotify.com/track/2Gj8BVWfiqHTykY...,NaN
2,anh muốn mình như con thuyền kia lao vào em rồ...,anh muốn mình như con thuyền kia lao vào em rồ...,Ngắn,"Ngắn, MHee",Single,26/02/2023,26/02/2023,Mhee,"Ngắn, MHee",LOOPS Music,LOOPS Music,6B9UOFI7,https://zingmp3.vn/bai-hat/anh-muon-minh-nhu-c...,https://open.spotify.com/track/4doZRO6lNutfus7...,https://www.nhaccuatui.com/bai-hat/anh-muon-mi...
3,bình dân,Bình Dân,Ngắn,NaN,Single,03/09/2021,NaN,None,NaN,NaN,Yin Yang Media,NaN,NaN,https://open.spotify.com/track/3dhDxLqZScUdYDx...,https://www.nhaccuatui.com/bai-hat/binh-dan-ng...
4,bình dân,Bình Dân - Beat,Ngắn,NaN,Single,09/03/2021,NaN,None,NaN,NaN,Yin Yang Media,NaN,NaN,https://open.spotify.com/track/0eof9V6sNRlPtiQ...,https://www.nhaccuatui.com/bai-hat/binh-dan-ng...


In [28]:
df_merged.to_excel("_songZingMP3+Spotify+NCT.xlsx",index=False)

In [ ]:
# import pandas as pd
# from fuzzywuzzy import process

# # Hàm fuzzy matching để chuẩn hóa tiêu đề mà không bỏ dấu tiếng Việt
# def fuzzy_standardize_title(title, reference_titles, threshold=90):
#     if not title or pd.isna(title):
#         return ""
#     best_match = process.extractOne(title, reference_titles, score_cutoff=threshold)  # So khớp giữ nguyên dấu
#     return best_match[0] if best_match else title  # Nếu có tiêu đề giống, trả về tiêu đề chuẩn

# # Giả sử df_ và df_nct là hai DataFrame chứa danh sách bài hát từ các nguồn khác nhau
# # Không thay đổi dấu, ký tự đặc biệt hoặc viết hoa

# # Lấy danh sách tiêu đề duy nhất từ df_nct
# unique_titles_nct = df_nct["tracklist(danh sách bài hát)"].dropna().unique()

# # Áp dụng fuzzy matching để chuẩn hóa tiêu đề trên df_
# df_final["final_title"] = df_final["tracklist(danh sách bài hát)"].apply(lambda x: fuzzy_standardize_title(x, unique_titles_nct))

# # Gộp dữ liệu từ NCT vào df_
# df_merged_nct = pd.merge(df_final, df_nct, left_on="final_title", right_on="tracklist(danh sách bài hát)", how="outer", suffixes=("", "_NCT"))

# # Nếu thiếu album_name, thay bằng NCT
# df_merged_nct["album_name"] = df_merged_nct["album_name"].fillna(df_merged_nct["album_name_NCT"])

# # Nếu thiếu track title, thay bằng NCT
# df_merged_nct["tracklist(danh sách bài hát)"] = df_merged_nct["tracklist(danh sách bài hát)"].fillna(df_merged_nct["tracklist(danh sách bài hát)_NCT"])
# # df_merged_nct["Album artist (nghệ sĩ sở hữu album)*(ZingMP3)"] = df_merged_nct["Album artist (nghệ sĩ sở hữu album)*(ZingMP3)"].fillna(artist_name)
# # df_merged_nct["Album artist (nghệ sĩ sở hữu album)*(Spotify)"] = df_merged_nct["Album artist (nghệ sĩ sở hữu album)*(Spotify)"].fillna(artist_name)
# # Xóa các cột không cần thiết
# df_final = df_merged_nct.drop(columns=["album_name_NCT","final_title","tracklist(danh sách bài hát)_NCT"])

# # Loại bỏ các bản ghi trùng lặp
# df_final = df_final.drop_duplicates(subset=["tracklist(danh sách bài hát)", "album_name"])

# # Sắp xếp theo tên album
# df_final = df_final.sort_values(by="album_name", ascending=True)

# # Xuất ra file Excel
# df_final.to_excel("songSpot+ZingMP3+NCT.xlsx", index=False)

# # Hiển thị 5 dòng đầu tiên
# df_final.head()

,album_name,Album artist (nghệ sĩ sở hữu album)*(Spotify),Album artist (nghệ sĩ sở hữu album)*(ZingMP3),album_type,tracklist(danh sách bài hát),Ngày phát hành trên Spotify,Ngày phát hành trên ZingMP3,Song artist(nghệ sĩ tham gia bài hát)(Spotify),Song artist(nghệ sĩ tham gia bài hát)(ZingMP3),Cung cấp bởi(ZingMP3),...,album_score,matched_tracklist,tracklist_score,combined,matched_combined,match_score,Link bài hát,norm_album_name_NCT,norm_tracklist_NCT,combined_NCT
0,120Bpm (ft. Long Mộng Gà),Ngắn,NaN,Single,120Bpm (ft. Long Mộng Gà),10/03/2023,NaN,Antoneus Maximus,NaN,NaN,...,90.0,120Bpm,90.0,120Bpm (Ft. Long Mộng Gà) - 120Bpm (Ft. Long M...,120Bpm - 120Bpm,86.0,https://www.nhaccuatui.com/bai-hat/120bpm-ngan...,120Bpm,120Bpm,120Bpm - 120Bpm
1,9X,DLBlack,"DLblack, Ngắn, Mêk Team",Single,9X,04/05/2020,04/05/2020,"DLBlack, Mêk Team","DLblack, Ngắn, Mêk Team",MIXUS,...,45.0,NaN,45.0,9X - 9X,NaN,51.0,NaN,NaN,NaN,NaN
4,Bình Dân,Ngắn,"Ngắn, Monkieq",Single,Bình Dân,03/09/2021,10/09/2021,None,"Ngắn, Monkieq",MIXUS,...,100.0,Bình Dân,100.0,Bình Dân - Bình Dân,Bình Dân - Bình Dân,100.0,https://www.nhaccuatui.com/bai-hat/binh-dan-ng...,Bình Dân,Bình Dân,Bình Dân - Bình Dân
5,Bình Dân,Ngắn,"Ngắn, Monkieq",Single,Bình Dân - Beat,09/03/2021,10/09/2021,None,"Ngắn, Monkieq",MIXUS,...,100.0,Bình Dân,90.0,Bình Dân - Bình Dân - Beat,Bình Dân - Bình Dân,95.0,https://www.nhaccuatui.com/bai-hat/binh-dan-ng...,Bình Dân,Bình Dân,Bình Dân - Bình Dân
176,Bụi Thiên Thần,Ngắn,Ngắn,EP,Xa Nghìn Trùng Mây,27/07/2022,22/07/2022,None,Ngắn,LOOPS Music,...,90.0,Xa Nghìn Trùng Mây,100.0,Bụi Thiên Thần - Xa Nghìn Trùng Mây,Xa Nghìn Trùng Mây - Xa Nghìn Trùng Mây,95.0,https://www.nhaccuatui.com/bai-hat/xa-nghin-tr...,Xa Nghìn Trùng Mây,Xa Nghìn Trùng Mây,Xa Nghìn Trùng Mây - Xa Nghìn Trùng Mây
